***
***
# <h1 style="text-align: center; color: orange;">RECOMMENDER SYSTEM</h1>
***
***

 <h3 > 
 
 Los sistemas de recomendación nos ayudan a responder preguntas como ¿Qué película debería rentar? ¿Qué TV debería comprar?

 Lo primero que debemos responder es:
 ¿Qué es un sistema de recomendación?
 
 Se puede definir como una herramienta que trabaja con información y provee información de items en los que estemos interesados 

 ¿Por qué queremos un sistema de recomendación?
 
 Los sistemas de recomendación existen porque a las personas no les gusta perdaer tiempo, lo cual nos ayuda a ahorra tiempo en la busqueda y elección.
 </h3>



## 
<h3 style="color:orange"> ¿Cómo funcionan los sistema de recomendación?</h3>


<h3> 
Existen diversas formas de crear un sistema de recomendación, algunos de estos son: 

* CONTENT-BASED FILTERING (CBF)
* COLLABORATIVE- FILTERING (CF)
</h3>

##
<h3 style="color:orange">  CONTENT-BASED FILTERING (CBF) </h3>

<h3> 
Está construido bajo el paradigma: "muestrame más de lo mismo que me ha gustado". Por lo cual este enfoque podrá recomentar items que son similares a los que el cliente le ha gustado y las recomendaciones están basadas en las descripciones de los item y las preferencias del perfil del cliente.  El cálculo de la similitud entre los items es lo más importante de esté método y se basa en el contenido de los items.

Cuando tratamos con información textual como noticias o libros, se usa el algoritmo *tf-idf* (frequency-inverse document frequency) que representa a una estadística númerica que nos indica la importancia de la palabra para el documento en la colección del corpus (conjunto grande de texto).
</h3>


##
<h3 style="color:orange">  COLLABORATIVE FILTERING (CF) </h3>

<h3> 
CF están construidos con el paradigma: "Dime que es lo más popular entre los usuarios con gustos similares". Una hipotesis importante detrás de esté recomendador es que los usuarios similares le gustan los mismos items.
Este enfoque se basa en recolectar y analizar un gran número de datos relacionados con el compartamiento de los usuarios, una vez analizados se predice que podría gustarle con base en la similitud de otros usuarios.

Una ventaja de este método es que no es necesarion "entender"  que item se está recomendanddo. Una desventaja es cuando el sistema no tiene suficiente información del usuario para realizar una inferencia.

Existen dos tipos de recomendadores para CF: 

* USER-BASED: Se encuentran a usuarios similares a mi y se recomienda que es lo que a ellos les gusta. En este método, dado un usuario U, primero encontramos a otros usuarios, que hayan ranquead de forma similar a U y luego se cacula la predicción para U.
* ITEM-BASED: Se encuentran los items similares que previamente al usuario le gustó.Y el item basado en CF. Se construye una matriz de items, que ayuda a identificar la relación entre items, después se usa la matriz y la información del usuario U, se realiza la inferencia del gusto del usuario. Este enfoque es útil para casos en los que si un cliente compra X tambien compra Y.
</h3>



##
<h3 style="color:orange">  EVALUATION RECOMMENDERS </h3>

<h3> 
La forma más común de evualuar un sistema de recomendación es a través de la predicción, la capacidad de predecir las elecciones de los usuarios. Algunas métricas comunes son RMSE (ROOT MEAN SQUARE ERROR), precisión, recall o ROC.
</h3>


##
<h3 style="color:orange"> PRACTICAL CASE </h3>

<h3> 
El dataset MovieLens es una colección de calificaciones a perliculas a partir de cientos de usuarios , un trabajo de la Universidad de Minnesota, el dataset original consta de 32M de registros y 2M de aplicaciones de etiquetas aplicadas a 87K películas por 200k usuarios. Nosotros trabajaremos con un dataset más reducido de 1M de registros, aproximadamente 3.9K películas y 6K usuarios que se unieron en el año 2000.
</h3>

In [ ]:
import pandas as pd
import numpy as np
import math
import matplotlib.pylab as plt
from math import isnan
from tqdm import tqdm 

from scipy.stats import pearsonr
from scipy.spatial.distance import euclidean

In [ ]:
# tags = pd.read_csv("./ml-32m/tags.csv")
# ratings = pd.read_csv("./ml-32m/ratings.csv")
# movies = pd.read_csv("./ml-32m/movies.csv")
# links = pd.read_csv("./ml-32m/links.csv")

In [ ]:
# path = "./ml-32m/"

# user data
u_cols = ["user_id","age","sex","occupation","zip_code"]
users = pd.read_csv("./ml-100k/u.user", sep = "|", names = u_cols ,engine='python')

# rating data
r_cols = ["user_id","movie_id","rating","unix_timestamp"]
ratings = pd.read_csv("./ml-100k/u.data" , sep = "\t", names = r_cols , engine='python')

# movie data
m_cols = ["movie_id","title","release_date"]
movies = pd.read_csv("./ml-100k/u.item", sep = "|", names = m_cols ,usecols=range(3) , engine='python', encoding="latin-1")


In [ ]:
data = pd.merge(pd.merge(users,ratings, on= "user_id" , how = "inner"),movies, on="movie_id", how = "inner")

print("La tabla users tiene: ", data.user_id.nunique() , " usuarios únicos.")
print("La tabla ratings tiene: ", data.shape[0] , " ratings.")
print("La tabla movies tiene: ", data.movie_id.nunique() , " películas únicas.")

In [ ]:
data = data[['user_id','title', 'movie_id','rating']]

In [ ]:
data.shape[0]/data.user_id.nunique()


##
<h3 style="color:orange"> USER-BASED COLLABORATIVE FILTERING </h3>

<h3> 
En el orden que definimos tenemos 

1. prediction function. 
2. user similarity function.
3. evaluation function.
</h3>

##
<h3 style="color:orange"> PREDICTION FUNCTION </h3>
<h3> 
La función de predicción atrás del modelo CF se basa en los ratings de las películas de usuarios similares.
</h3>

* p pertenece al conjunto de películas P.
* a un usuario dado.
* B conjunto de usuarios.

$$pred(a,p) = \frac{\sum_{b \in B}{sim(a,b)*(r_{b,p})}}{\sum_{b \in B}{sim(a,b)}}$$

<h3> 
donde sim(a,b) sería la similitud entre los usuarios a y b, B está dado por el conjunto de usuarios que ya vieron la película p, r_{b,p} es el ranking de p dado por el usuario b. 
</h3>

##
<h3 style="color:orange"> USER SIMILARITY </h3>

<h3> 
El cálculo de la similitud entre los items es uno de los pasos criticos en el algoritmo CF. La idea básica detrás de la similitud entre los usuarios a y b, es donde primero podemos aislar  el conjunto P de los items que fueron calificados por ambos usuarios y después aplicar la función de similitud.
</h3>

In [ ]:
## user 1 
data_user_1 = data[data.user_id==1]

## user 6
data_user_2 = data[data.user_id==6]

## common movies from user 1 and 6
common_movies = set(data_user_1.movie_id).intersection(data_user_2.movie_id)

print("Número de películas en común:", len(common_movies))

mask = data_user_1.movie_id.isin(common_movies)
data_user_1 = data_user_1[mask]
print(data_user_1[["title","rating"]])

mask = data_user_2.movie_id.isin(common_movies)
data_user_2 = data_user_2[mask]
print(data_user_2[["title","rating"]])


<h3> 
Una vez obtenidos las calificaciones de las películas en común de dos usuarios, podemos calculas la similitud, estas son formas comunes de calcularla: 
</h3>

<ul>
<li>Distancia Euclidiana</li>

$$sim(a,b) = \frac{1}{1+\sqrt{\sum_{p \in P}{(r_{a,p} - r_{b,p})^2}}}$$
<br>
<li>Correlación de Pearson</li>

$$sim(a,b) = \frac{\sum_{p\in P} (r_{a,p}-\bar{r_a})(r_{b,p}-\bar{r_b})}{\sqrt{\sum_{p \in P}(r_{a,p}-\bar{r_a})²}\sqrt{\sum_{p \in P}(r_{b,p}-\bar{r_b})²}}$$
<br>
<li>Distancia coseno (Similitud coseno) </li>

$$ sim(a,b) = \frac{\vec{a}· \vec{b}}{|\vec{a}| * |\vec{b}|}$$
<br>
</ul>

<br>
Donde: 

* $sim(a,b)$ es la similitud entre el usuario "a" y el "b".
* $P$ es el conjunto de peliculas calificadas en común por los usuarios "a" y "b"
* $r_{a,p}$ es la calificación de la película "p" por el usuario "a"
* $\bar{r_a}$ es la media de las calificaciones por el usuario "a"

<br>


<h4>Algunos problemas</h4>
<li>La correlación de pearson es mejor que la distancia euclidiana ya que se basa más en las calificaciones que en los valores.</li>
<li>La distanicia coseno es usualmente usada cuando los datos son binarios o unitarios.</li>
</ul>

In [ ]:
def user_sim(df, user_1, user_2, 
                    min_common_items = 3, 
                    method = 'pearson'):
    # GET MOVIES OF USER1
    mov_usr1 = df[df['user_id'] == user_1 ]
    # GET MOVIES OF USER2
    mov_usr2 = df[df['user_id'] == user_2 ]
    
    # FIND SHARED FILMS
    df_shared = pd.merge(mov_usr1, mov_usr2, on='movie_id')  
    
    # If there is no enough common items to comput similarity
    if(df_shared.shape[0]<=min_common_items):
        return 0
    
    if method =='pearson':
        res = pearsonr(df_shared['rating_x'],df_shared['rating_y'])[0]
        if(np.isnan(res)):
            return 0
        return res
    
    elif method =='euclidean':
        return 1.0/(1.0+euclidean(df_shared['rating_x'],
                                  df_shared['rating_y'])) 
    else:
        print("method not defined")
        return 0
    


In [ ]:
print("Euclidean Similarity",
      user_sim(data,1,8,method = 'euclidean') )
print("Pearson Similarity",user_sim(data,1,8) )

In [ ]:
print("Euclidean Similarity",
      user_sim(data,1,31, method = 'euclidean') )
print("Pearson Similarity",user_sim(data,1,31) )

In [ ]:
user_id_1, user_id_2 = 3, 8

movies_user1=data[data['user_id'] ==user_id_1 ][['user_id','movie_id','rating']]
movies_user2=data[data['user_id'] ==user_id_2 ][['user_id','movie_id','rating']]
    
# FIND SHARED FILMS
rep=pd.merge(movies_user1 ,movies_user2,on='movie_id')
x= rep.rating_x + np.random.normal(loc=0.0, scale=0.1,size=len(rep.rating_x))
y= rep.rating_y +np.random.normal(loc=0.0, scale=0.1,size=len(rep.rating_y))
    
a=rep.groupby(['rating_x', 'rating_y']).size()
x=[]
y=[]
s=[]
for item,b in a.items():
    x.append(item[0])
    y.append(item[1])
    s.append(b*30.)

fig = plt.figure(figsize=(6,4))
plt.scatter(x,y, s=s)
plt.xlabel('Rating User 3')
plt.ylabel('Rating User '+str(8))
plt.axis([0.5,5.5,0.5,5.5])
# plt.savefig("corre18.png",dpi= 300, bbox_inches='tight')
plt.show()

In [ ]:
user_id_1, user_id_2 = 3, 31
movies_user1=data[data['user_id'] ==user_id_1 ][['user_id','movie_id','rating']]
movies_user2=data[data['user_id'] ==user_id_2 ][['user_id','movie_id','rating']]
    
# FIND SHARED FILMS
rep=pd.merge(movies_user1 ,movies_user2,on='movie_id')
x= rep.rating_x + np.random.normal(loc=0.0, scale=0.1,size=len(rep.rating_x))
y= rep.rating_y +np.random.normal(loc=0.0, scale=0.1,size=len(rep.rating_y))
    
a=rep.groupby(['rating_x', 'rating_y']).size()
x=[]
y=[]
s=[]
for item,b in a.items():
    x.append(item[0])
    y.append(item[1])
    s.append(b*30.)

fig = plt.figure(figsize=(6,4))
plt.scatter(x,y, s=s)
plt.xlabel('Rating User 3')
plt.ylabel('Rating User '+str(31))
plt.axis([0.5,5.5,0.5,5.5])
# plt.savefig("corre131.png",dpi= 300, bbox_inches='tight')
plt.show()

<h3>EVALUACIÓN DEL MODELO</h3>

Dividiremos el datasets principal en dos diferente, uno llamado $X_{train}$ con el 80\% de los datos y otro llamado $X_{test}$ con el 20\% restante.


In [ ]:
def assign_to_set(df):
    sampled_ids = np.random.choice(
        df.index,
        size=np.int64(np.ceil(df.index.size * 0.2)),
        replace=False)
    df.loc[sampled_ids, 'for_testing'] = True
    return df

def create_train_test(data, key = 'user_id'):
    data['for_testing'] = False
    grouped = data.groupby(key, group_keys=False).apply(assign_to_set,include_groups=False)
    # dataframe used to train our model
    data_train = data[grouped.for_testing == False]
    # dataframe used to evaluate our model
    data_test = data[grouped.for_testing == True]
    return data_train, data_test


X_train, X_test =  create_train_test(data)

print("#Training samples = ",X_train.shape[0])
print("#Test samples = ",X_test.shape[0])
print('#Users =', X_train.user_id.nunique())
print('#Movies =',X_train.movie_id.nunique())

##
<h3 style="color:orange"> MÉTRICAS DE EVALUACIÓN</h3>

In [ ]:
def compute_rmse(y_pred, y_true):
    """ Compute Root Mean Squared Error. """
    return np.sqrt(np.mean(np.power(y_pred - y_true, 2)))

def precision(recommended_items, relevant_items):
    is_relevant = np.in1d(recommended_items, 
                          relevant_items, 
                          assume_unique=True)
    
    precision = np.sum(is_relevant, dtype=np.float32) / len(is_relevant)
    
    return precision

def recall(recommended_items, relevant_items):  
    is_relevant = np.in1d(recommended_items, 
                          relevant_items, 
                          assume_unique=True)
    
    recall = np.sum(is_relevant, dtype=np.float32) / relevant_items.shape[0]
    
    return recall

def AP(recommended_items, relevant_items):
    is_relevant = np.in1d(recommended_items, 
                          relevant_items, 
                          assume_unique=True)
    
    # Cumulative sum: precision at 1, at 2, at 3 ...
    p_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(is_relevant.shape[0]))
    ap_score = np.sum(p_at_k) / np.min([relevant_items.shape[0], is_relevant.shape[0]])

    return ap_score



In [ ]:
def evaluate(rec_object, train, test, 
             at = 20, thr_relevant= 4):
    """ Perfomance evaluation """
    
    # RMSE evaluation
    ids_to_estimate = zip(test.user_id, test.movie_id)
    y_estimated = np.array([rec_object.predict_score(u,i) 
                          if u in train.user_id else 3 
                          for (u,i) in ids_to_estimate ])
    
    y_real = test.rating.values
    rmse = compute_rmse(y_estimated, y_real)
    
    print("RMSE: {:.4f}".format(rmse))
          
    cumulative_precision = 0.0
    cumulative_recall = 0.0
    cumulative_AP = 0.0

    num_eval = 0

    for user_id in tqdm(test.user_id.unique()):

        relevant_items = test[(test.user_id==user_id )&( test.rating>=thr_relevant)].movie_id.values

        if len(relevant_items)>0:

            recommended_items = rec_object.predict_top(user_id, at=at)
            num_eval+=1

            cumulative_precision += precision(recommended_items, relevant_items)
            cumulative_recall += recall(recommended_items, relevant_items)
            cumulative_AP += AP(recommended_items, relevant_items)

    cumulative_precision /= num_eval
    cumulative_recall /= num_eval
    MAP = cumulative_AP / num_eval

    print("Precision = {:.4f}, Recall = {:.4f}, MAP = {:.4f}".format(
        cumulative_precision, cumulative_recall, MAP)) 
    

##
<h3 style="color:orange"> NAIVE RECOMMENDER SYSTEM</h3>

<h3>
Primero veamos los resultados de un sistema de recomendación aleatorio (random).
</h3>


In [ ]:
class RandomRecommender():

    def fit(self, train):
        self.items = train.title.unique()
    
    def predict_score(self, user_id, movie_id):
        '''Given a user_id and movie_id predict its score'''
        return np.random.uniform(1,5)
    
    def predict_top(self, user_id, at=5):
        '''Given a user_id predicts its top 'at' movies'''
        recommended_items = np.random.choice(self.items, at)

        return recommended_items

In [ ]:
random_model = RandomRecommender()
random_model.fit(X_train)

In [ ]:
evaluate(random_model,X_train,X_test, at = 50)

<h3>
El modelo aleatorio nos da un RMSE de 1.63 y las demás métricas del 0%
</h3>

<h3>
Ahora construimos el sistema de recomendación con CF, 
</h3>

In [ ]:
class CollaborativeFiltering:
    """ Collaborative filtering model """
    
    def __init__(self, similarity = 'pearson'):
        '''Constructor'''
        self.sim_method=similarity
        self.sim = {}

    def fit(self, train):
        '''Prepare data structures for estimation'''
        self.train = train
        allUsers = set(self.train['user_id'])
        
        # Create a dictionary with user, movie, rating
        self.seen_movies = {user: {} for user in allUsers}
            
        for user in allUsers:
            user_ratings = train[train.user_id == user][['movie_id', 'rating']]
            self.seen_movies[user] = dict(zip(user_ratings['movie_id'], user_ratings['rating']))
    
        for usr_1 in tqdm(allUsers):
            self.sim.setdefault(usr_1, {})
            a = self.train[self.train.user_id == usr_1][['movie_id']]
            data_reduced = pd.merge(self.train, a , on='movie_id') 
            
            for usr_2 in allUsers:
                # Avoid comparing a user with themselves
                if usr_1 == usr_2: 
                    continue      
                self.sim.setdefault(usr_2, {})
                if(usr_1 in self.sim[usr_2]):
                    continue # since is a simetric matrix
                sim = user_sim(data_reduced, usr_1, usr_2, 
                               method = self.sim_method)
                if(sim < 0):
                    self.sim[usr_1][usr_2]=0
                    self.sim[usr_2][usr_1]=0
                else:
                    self.sim[usr_1][usr_2]=sim
                    self.sim[usr_2][usr_1]=sim
        
        
    def predict_score(self, usr_id, movie_id):
        ''' Given a user_id and movie_id it predicts its score'''
        seen = self.train[self.train.movie_id == movie_id]
        rating_num, rating_den = 0.0, 0.0
        allUsers = set(seen['user_id'])
        # Iterate through all users who have seen the movie
        for other in allUsers:
            if usr_id == other:
                continue  # Skip the current user, as self-similarity is not needed
            
            similarity = self.sim[usr_id][other]
            rating = float(self.seen_movies[other][movie_id])
            
            rating_num += similarity * rating
            rating_den += similarity
            
        # If the denominator is zero (no similar users), handle the case
        if rating_den == 0: 
            if self.train.rating[self.train['movie_id'] == movie_id].mean()>0:
                # return the mean movie rating if there is no similar for the computation
                return self.train.rating[self.train['movie_id'] == movie_id].mean()
            else:# return mean user rating 
                return self.train.rating[self.train['user_id'] == usr_id].mean()
       
        # return the predicted score
        return rating_num/rating_den

    def predict_top(self, usr_id, at=20, remove_seen=True):
        '''Given a usr_id predict its top 'at' movies'''
        # Get the movies already seen by the user
        seen_items = set(self.train[self.train.user_id == usr_id].movie_id.values)

        # Get the set of unseen items
        unseen_items = set(self.train.movie_id.values) - seen_items

        # Generate predictions for unseen items
        predictions = [(item_id, self.predict_score(usr_id, item_id)) for item_id in unseen_items]

        # Sort predictions by score in descending order and select the top 'at' items
        sorted_predictions = sorted(predictions, key=lambda x: x[1], reverse=True)[:at]

        # Extract the item IDs from the sorted predictions
        top_items = [item_id for item_id, _ in sorted_predictions]
        
        return top_items

In [ ]:
cf_model = CollaborativeFiltering()
cf_model.fit(X_train)
cf_model.predict_score(usr_id = 2, movie_id = 1)

In [ ]:
# se tarda en procesar 8 horas
# evaluate(cf_model,X_train,X_test, at = 50)

# RMSE: 1.0630

# Precision = 0.0197, Recall = 0.0687, MAP = 0.0032

<h3> 
Sabemos que las calificaciones de las personas no tienen el mismos compartamiento que los críticos, algunos califican con mayor o menor puntuación, la siguiente función toma en cuenta la media del usuarios.

$$pred(a,p) = \bar{r_a} + \frac{\sum_{b \in N}{sim(a,b)*(r_{b,p}-\bar{r_b})}}{\sum_{b \in N}{sim(a,b)}}$$

In [ ]:
class CollaborativeFiltering_Ex1(CollaborativeFiltering):
    def predict_score(self, usr_id, movie_id, N = 10):
        ''' Given a user_id and movie_id it predicts its score'''
        seen = self.train[self.train['movie_id'] ==movie_id]
        rating_num, rating_den = 0.0, 0.0
        allUsers = set(seen['user_id'])
        
        for other in allUsers:
            if usr_id == other:
                continue  # Skip the current user, as self-similarity is not needed
            similarity = self.sim[usr_id][other]
            rating = float(self.seen_movies[other][movie_id])
            mean_user_rating = np.mean([self.seen_movies[other][key] 
                                        for key in self.seen_movies[other]])
            rating_num += similarity * float(rating - mean_user_rating)  
            rating_den += similarity
        if rating_den == 0: 
            if self.train.rating[self.train['movie_id'] == movie_id].mean()>0:
                # return the mean movie rating if there is no similar for the computation
                return self.train.rating[self.train['movie_id'] == movie_id].mean()
            else:
                # else return mean user rating 
                return self.train.rating[self.train['user_id'] == usr_id].mean()
        mean_rating_user = self.train[self.train.user_id==usr_id].rating.mean()
        return mean_rating_user + rating_num/rating_den

In [ ]:
cf_model_v2 = CollaborativeFiltering_Ex1()
cf_model_v2.fit(X_train)
cf_model_v2.predict_score(usr_id = 2, movie_id = 1)

In [ ]:
# evaluate(cf_model_v2, X_train, X_test, at = 50)